# Connect with `City` Agent for remote SMC

In [1]:
%%javascript
document.title="Manufacturer1 Agent"

<IPython.core.display.Javascript object>

### Imports

In [13]:
from aries_cloudcontroller import AriesAgentController
import os
from termcolor import colored
from pprintpp import pprint

### Initialise the Agent Controller

In [3]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://manufacturer1-agent:3021 and an api key of adminApiKey


### Start a Webhook Server

In [4]:
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

await agent_controller.init_webhook_server(webhook_host, webhook_port)

print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


## Register Agent Event Listeners

You can see some examples within the webhook_listeners recipe. Copy any relevant cells across and customise as needed.

In [5]:
listeners = []

# Receive connection messages
def connections_handler(payload):
    state = payload['state']
    connection_id = payload["connection_id"]
    their_role = payload["their_role"]
    routing_state = payload["routing_state"]
    
    print("----------------------------------------------------------")
    print("Connection Webhook Event Received")
    print("Connection ID : ", connection_id)
    print("State : ", state)
    print("Routing State : ", routing_state)
    print("Their Role : ", their_role)
    print("----------------------------------------------------------")

    if state == "invitation":
        # Your business logic
        print("invitation")
    elif state == "request":
        # Your business logic
        print("request")

    elif state == "response":
        # Your business logic
        print("response")
    elif state == "active":
        # Your business logic
        print(colored("Connection ID: {0} is now active.".format(connection_id), "green", attrs=["bold"]))



connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

listeners.append(connection_listener)


def prover_proof_handler(payload):
    role = payload["role"]
    connection_id = payload["connection_id"]
    pres_ex_id = payload["presentation_exchange_id"]
    state = payload["state"]
    print("\n---------------------------------------------------------------------\n")
    print("Handle present-proof")
    print("Connection ID : ", connection_id)
    print("Presentation Exchange ID : ", pres_ex_id)
    print("Protocol State : ", state)
    print("Agent Role : ", role)
    print("Initiator : ", payload["initiator"])
    print("\n---------------------------------------------------------------------\n")
    
    
    if state == "request_received":
        presentation_request = payload["presentation_request"]
        print("Recieved Presentation Request\n")
        print("\nRequested Attributes - Note the restrictions. These limit the credentials we could respond with\n")
        print(presentation_request["requested_attributes"])
    elif state == "presentation_sent":
        print("Presentation sent\n")
        
    elif state == "presentation_acked":
        print("Presentation has been acknowledged by the Issuer")
        
prover_listener = {
    "topic": "present_proof",
    "handler": prover_proof_handler
}

listeners.append(prover_listener)


agent_controller.register_listeners(listeners)

## Accept Invitation

Copy an invitation object from another agent playing the role inviter (see the inviter_template recipe)

In [6]:
invitation = {
    '@id': '808061b1-42a3-495b-b8c8-c9c15e9c6e05',
    '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
    'label': 'City',
    'recipientKeys': ['3XPvTHWspTMf1wWWtsaFeviaEPaejD5c1Z3XRwRys5bA'],
    'serviceEndpoint': 'https://54a5a5d5a57e.ngrok.io',
}

In [7]:
auto_accept="true"
alias=None

invite_response = await agent_controller.connections.receive_invitation(invitation, alias, auto_accept)
connection_id = invite_response["connection_id"]

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  34d38d37-2418-4e1f-ad08-e103030a15a6
State :  invitation
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
invitation
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  34d38d37-2418-4e1f-ad08-e103030a15a6
State :  request
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
request
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  34d38d37-2418-4e1f-ad08-e103030a15a6
State :  response
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
response
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  34d38d37-2418-4e1f-ad08-e103030a15a6
State :  active
Rout

## Optional: Send Proposal

Propose a presentation to a verifier

In [ ]:
# TODO: Example proposal object below

# proposal_object = {
#   "auto_present": true,
#   "comment": "string",
#   "connection_id": "3fa85f64-5717-4562-b3fc-2c963f66afa6",
#   "presentation_proposal": {
#     "@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/presentation-preview",
#     "attributes": [
#       {
#         "cred_def_id": "WgWxqztrNooG92RXvxSTWv:3:CL:20:tag",
#         "mime-type": "image/jpeg",
#         "name": "favourite_drink",
#         "referent": "0",
#         "value": "martini"
#       }
#     ],
#     "predicates": [
#       {
#         "cred_def_id": "WgWxqztrNooG92RXvxSTWv:3:CL:20:tag",
#         "name": "high_score",
#         "predicate": ">=",
#         "threshold": 0
#       }
#     ]
#   },
#   "trace": false
# }

# proposal_response = await agent_controller.proofs.send_proposal(proposal_object)

## Fetch Presentation Records

Before you can present a presentation, you must identify the presentation record which you wish to respond to with a presentation. This could also be done through the present_proof listeners which have access to a presentation record in the payload.

In [8]:
# Optional Query parameters
verifier_connection_id = connection_id
thread_id=None
state = "request_received"
role = "prover"

proof_records_response = await agent_controller.proofs.get_records(verifier_connection_id, thread_id, state, role)

# We fetch the first record from the response. You may want to customise this further
presentation_record = proof_records_response["results"][0]
presentation_exchange_id = presentation_record["presentation_exchange_id"]

In [14]:
pprint(presentation_record)

{
    'connection_id': '34d38d37-2418-4e1f-ad08-e103030a15a6',
    'created_at': '2021-08-05 07:47:08.469712Z',
    'initiator': 'external',
    'presentation_exchange_id': 'e2ad02fb-8bbe-4468-baae-3c0f89549a38',
    'presentation_request': {
        'name': 'Name of Proof Request',
        'non_revoked': {'to': 1628149482},
        'nonce': '471517040101429148699515',
        'requested_attributes': {
            '0_isManufacturer_uuid': {
                'name': 'isManufacturer',
                'restrictions': [
                    {
                        'issuer_did': 'KxYXF6vfTXMoBNg2uGt79g',
                        'schema_id': 'BQmsjXtsjzzmTtdwQg5Ftn:2:certify-manufacturer:0.0.1',
                    },
                ],
            },
        },
        'requested_predicates': {},
        'version': '1.0',
    },
    'presentation_request_dict': {
        '@id': '83bbb9db-0e62-42a3-aabe-9ec961154850',
        '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/re

## Search For Available Credentials to Construct Presentation From

The presentation record can be used to query your agents wallet and return all credentials that could be used to construct valid presentation

In [16]:
# select credentials to provide for the proof
credentials = await agent_controller.proofs.get_presentation_credentials(presentation_exchange_id)
print("Credentials stored that could be used to satisfy the request. In some situations you applications may have a choice which credential to reveal\n")

attribute_by_reft = {}
revealed = {}
self_attested = {}
predicates = {}


# Note we are working on a friendlier api to abstract this away

if credentials:
    for credential in credentials:

        for attribute_reft in credential["presentation_referents"]:
            if attribute_reft not in attribute_by_reft:
                attribute_by_reft[attribute_reft] = credential

for (key, value) in attribute_by_reft.items():
    print(f"Attribute {presentation_record['presentation_request']['requested_attributes'][key]} can be satisfied by Credential with Referent -- {value['cred_info']['referent']}")
                
for attribute_reft in presentation_record["presentation_request"]["requested_attributes"]:
    if attribute_reft in attribute_by_reft:
        revealed[attribute_reft] = {
            "cred_id": attribute_by_reft[attribute_reft]["cred_info"][
                "referent"
            ],
            "revealed": True,
        }


print("\nGenerate the proof")
presentation = {
    "requested_predicates": predicates,
    "requested_attributes": revealed,
    "self_attested_attributes": self_attested,
}
print(presentation)

Credentials stored that could be used to satisfy the request. In some situations you applications may have a choice which credential to reveal


Generate the proof
{'requested_predicates': {}, 'requested_attributes': {}, 'self_attested_attributes': {}}


## Send Presentation

A presentation is sent in represent to a presentation record that has previously been created.

In [17]:
presentation_response = await agent_controller.proofs.send_presentation(presentation_exchange_id, presentation)

ERROR:aiohttp.server:Error handling request
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/aiohttp/web_protocol.py", line 422, in _handle_request
    resp = await self._request_handler(request)
  File "/opt/conda/lib/python3.9/site-packages/aiohttp/web_app.py", line 499, in _handle
    resp = await handler(request)
  File "/opt/conda/lib/python3.9/site-packages/aries_cloudcontroller/aries_webhook_server.py", line 85, in _receive_webhook
    await self._handle_webhook(wallet_id, topic, payload)
  File "/opt/conda/lib/python3.9/site-packages/aries_cloudcontroller/aries_webhook_server.py", line 108, in _handle_webhook
    pub.sendMessage(pub_topic_path, payload=payload)
  File "/opt/conda/lib/python3.9/site-packages/pubsub/core/publisher.py", line 216, in sendMessage
    topicObj.publish(**msgData)
  File "/opt/conda/lib/python3.9/site-packages/pubsub/core/topicobj.py", line 452, in publish
    self.__sendMessage(msgData, topicObj, msgDataSubset)
  File 

ClientResponseError: 400, message='Error when constructing proof: Error: Invalid structure. Caused by: Requested Credentials validation failed: `self_attested_attributes` and `requested_attributes` and `requested_predicates` are empty. CommonInvalidStructure.', url=URL('http://manufacturer1-agent:3021/present-proof/records/43d8ab93-4798-4994-a24f-8546c6b46bd2/send-presentation')

ERROR:aiohttp.server:Error handling request
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/aiohttp/web_protocol.py", line 422, in _handle_request
    resp = await self._request_handler(request)
  File "/opt/conda/lib/python3.9/site-packages/aiohttp/web_app.py", line 499, in _handle
    resp = await handler(request)
  File "/opt/conda/lib/python3.9/site-packages/aries_cloudcontroller/aries_webhook_server.py", line 85, in _receive_webhook
    await self._handle_webhook(wallet_id, topic, payload)
  File "/opt/conda/lib/python3.9/site-packages/aries_cloudcontroller/aries_webhook_server.py", line 108, in _handle_webhook
    pub.sendMessage(pub_topic_path, payload=payload)
  File "/opt/conda/lib/python3.9/site-packages/pubsub/core/publisher.py", line 216, in sendMessage
    topicObj.publish(**msgData)
  File "/opt/conda/lib/python3.9/site-packages/pubsub/core/topicobj.py", line 452, in publish
    self.__sendMessage(msgData, topicObj, msgDataSubset)
  File 

## Your Own Business Logic

Now you should have an established, active connection you can write any custom logic you want to engage with protocols with the connection

In [ ]:
## Custom Logic

## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [ ]:
await agent_controller.terminate()